In [ ]:
# a. Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# b. Load CIFAR-10 dataset and preprocess images
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    'cifar-10-img/train', target_size=(32, 32), batch_size=32, class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    'cifar-10-img/test', target_size=(32, 32), batch_size=32, class_mode='categorical'
)

# c. Load pre-trained VGG16 model with local weights file and add custom layers
base_model = VGG16(weights=None, include_top=False, input_shape=(32, 32, 3))
base_model.load_weights(r"vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")
base_model.trainable = False  # Freeze base model layers

# Custom classifier on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# d. Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=test_generator, epochs=10)

# e. Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# f. Predict class for a new image
new_img_path = 'cifar-10-img/test/bird/7.png'
img = tf.keras.preprocessing.image.load_img(new_img_path, target_size=(32, 32))
img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)[0]
class_labels = {v: k for k, v in train_generator.class_indices.items()}
print(f"Predicted class for the image: {class_labels[predicted_class]}")

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()


Found 40079 images belonging to 10 classes.
Found 9921 images belonging to 10 classes.



Epoch 1/10


1253/1253 [==============================] - 116s 92ms/step - loss: 1.3964 - accuracy: 0.5127 - val_loss: 1.2515 - val_accuracy: 0.5670
Epoch 2/10
1253/1253 [==============================] - 113s 90ms/step - loss: 1.2047 - accuracy: 0.5810 - val_loss: 1.2081 - val_accuracy: 0.5771
Epoch 3/10
1253/1253 [==============================] - 136s 108ms/step - loss: 1.1460 - accuracy: 0.5991 - val_loss: 1.1883 - val_accuracy: 0.5809
Epoch 4/10
1253/1253 [==============================] - ETA: 0s - loss: 1.1085 - accuracy: 0.6132